In [1]:
import json

In [2]:
#Import input data
jsonFile = open('/content/algo_input.json',)
jsonObject = json.load(jsonFile)

In [3]:
#Assign variables
vehicles = jsonObject["vehicles"]
jobs = jsonObject["jobs"]
matrix = jsonObject["matrix"]

In [4]:
#Finding best duration
def findBestDuration(vehicles,jobs):
  arr = []
  for vehicle in vehicles:
    for job in jobs:
      i = vehicle["start_index"]
      j = job["location_index"]
      json = {"v_id": vehicle["id"], "j_id": job["id"], "start_i":i,"location_i":j, "d": matrix[i][j]}
      arr.append(json)
  arr.sort(key = lambda json: json['d'], reverse=False)
  return arr[0]

In [5]:
#Finding best vehicle-job mathing routes
def findBestMatch(vehicles,jobs):
  arr = [] 
  total_delivery_duration = 0
  
  for i in range(len(vehicles)*len(jobs)):
    if len(jobs) <= 0: 
      break
      
    #find best duration for vehicles-jobs
    bestDurationOutput = findBestDuration(vehicles,jobs)

    #total delivery duration update
    total_delivery_duration += bestDurationOutput["d"] 
      
    #output json array
    arr.append(bestDurationOutput)

    #update vehicle start_index in vehicles
    list(filter(lambda x: x['id'] == bestDurationOutput["v_id"], vehicles))[0]["start_index"] = bestDurationOutput["location_i"]
      
    #delete completed job from jobs
    for j in range(len(jobs)):
      if jobs[j]["id"] == bestDurationOutput["j_id"]:
        del jobs[j]
        break

  return arr, total_delivery_duration

In [6]:
#Create output json data
from itertools import groupby
arr,total_delivery_duration = findBestMatch(vehicles,jobs)
def groupByOutputData(arr):
  res = []
  delivery_duration = 0
  key_func = lambda k: k['v_id']

  for k, g in groupby(sorted(arr, key=key_func), key=key_func):
      obj = { k : {'jobs': [] ,'delivery_duration': '' }}
      for group in g:
        delivery_duration += group['d']
        obj[k]['delivery_duration'] = delivery_duration
        obj[k]['jobs'].append(str(group['j_id']))
      res.append(obj)
      delivery_duration = 0 
  return res

In [7]:
#Output json
routes = groupByOutputData(arr)
routes

[{1: {'delivery_duration': 3516, 'jobs': ['7', '1', '2', '5', '6', '3']}},
 {3: {'delivery_duration': 226, 'jobs': ['4']}}]

In [8]:
#Empty routes added in json output 
vehicle_ids = [v["id"] for v in vehicles]
route_ids = [key for r in routes for key in r.keys()]
empty_routes = set(vehicle_ids) - set(route_ids)
if len(empty_routes) > 0 :
  for i in empty_routes:
    routes.append({i: {'jobs': [],'delivery_duration': 0}})

In [9]:
#Json Output
jsonOutput = {"total_delivery_duration" : total_delivery_duration, "routes": routes}
jsonOutput

{'routes': [{1: {'delivery_duration': 3516,
    'jobs': ['7', '1', '2', '5', '6', '3']}},
  {3: {'delivery_duration': 226, 'jobs': ['4']}},
  {2: {'delivery_duration': 0, 'jobs': []}}],
 'total_delivery_duration': 3742}

In [10]:
with open('/content/algo_output.json', 'w') as outfile:
    json.dump(jsonOutput, outfile)